# ADALA Quickstart

In this notebook, we are going to run through some of the common tasks for creating data labeling agents with ADALA. In this example, we're going to create a data labeling agent for a text classification task - labeling our text samples as either "Subjective or "Objective" statements. 

This agent will be LLM-based, so we will use [OpenAI's API](https://platform.openai.com/). You will to generate an API key and set it as an environment variable as follows: 

```
export OPENAI_API_KEY=your_openai_api_key
```

Now, let's begin. 

## Dataset Creation
First, let's use a dataset of product reviews stored in pandas dataframe. This will help us manage our data as we add more attributes, like predictions and labels for subjectivity and objectivity over time. 

In [1]:
import pandas as pd

df = pd.DataFrame([
    ["The mic is great.", "Subjective"],
    ["Will order from them again!", "Subjective"],
    ["Not loud enough and doesn't turn on like it should.", "Objective"],
    ["The phone doesn't seem to accept anything except CBR mp3s", "Objective"],
    ["All three broke within two months of use.", "Objective"]
], columns=["text", "ground_truth"])

df

,text,ground_truth
0,The mic is great.,Subjective
1,Will order from them again!,Subjective
2,Not loud enough and doesn't turn on like it sh...,Objective
3,The phone doesn't seem to accept anything exce...,Objective
4,All three broke within two months of use.,Objective


We instantiate Dataset that uses this pandas dataframe as a data source. Dataset object takes care of input data schema and data streaming:

In [2]:
from adala.datasets import DataFrameDataset

dataset = DataFrameDataset(df=df)

## Create Agent

To create Agent, we need to to define 2 things:

**Skills** - Agent's abilities are defined as _Skills_. Each agent can possess many different skills. In our case, this agent only has one labeling skill, to produce a classification of Subjective or Objective for a given piece of text.  To define this skill, we will leverage an LLM, passing it instructions and the set of labeles we expect to receive back. 

**Environment** - that is where the Agent receives ground truth signal to improve its skill. Since we already created ground truth dataset, we can simply refer to the column from the dataframe. In the real world scenario, you may consider using a different environment where ground truth signal can be obtained asynchoronously by gathering real human feedback during agent's learning phase.

In [9]:
from adala.agents import Agent
from adala.environments import BasicEnvironment
from adala.skills import ClassificationSkill
from adala.runtimes import OpenAIRuntime
from rich import print


agent = Agent(
    # define the agent's labeling skill that should classify text onto 2 categories
    skills=ClassificationSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective'],
        input_data_field='text'
    ),
    
    # basic environment extracts ground truth signal from the input records
    environment=BasicEnvironment(
        ground_truth_dataset=dataset,
        ground_truth_columns={'subjectivity_detection': 'ground_truth'}
    ),
    
    runtimes = {
        # You can specify your OPENAI API KEY here via `OpenAIRuntime(..., api_key='your-api-key')`
        'openai': OpenAIRuntime(model='gpt-3.5-turbo-instruct'),
    },
    default_runtime='openai',
    
    teacher_runtimes = {
      'openai-gpt3': OpenAIRuntime(model='gpt-3.5-turbo'),
      'openai-gpt4': OpenAIRuntime(model='gpt-4'),
    },
    
    # NOTE! If you don't have an access to gpt4 - replace it with "openai-gpt3"
    default_teacher_runtime='openai-gpt4'
)

print(agent)

Agent Instance

Environment: BasicEnvironment
Skills: subjectivity_detection
Runtimes: openai
Default Runtime: openai
Default Teacher Runtime: openai-gpt4

## Learning Agent

We will now let Agent learn from the ground truth. After every action, Agent returns its _Experience_, where it stores various observations like predicted data, errors, accuracy, etc.

In [10]:
ground_truth_signal = agent.learn(learning_iterations=3, accuracy_threshold=0.95)

Applying skill: subjectivity_detection

100%|████████████████████████████████| 5/5 [00:00<00:00, 45.97it/s]


=> Iteration #0: Comparing to ground truth, analyzing and improving ...

Comparing predictions to ground truth data ...

 text                     ground_truth   subjectivity_detection   score                    subjectivity_detecti…  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  The mic is great.        Subjective     Subjective               {'Subjective':           True                   
                                                                   -0.02697588099999997,                           
                                                                   'Objective':                                    
                                                                   -3.6262724}                                     
  Will order from them     Subjective     Subjective               {'Subjective':           True                   
  again!                                                           -0.11282212000000001,                           
                                                                   'Objective':                                    
                                                                   -2.2378219999999995}                            
  Not loud enough and      Objective      Subjective               {'Subjective':           False                  
  doesn't turn on like                                             -0.014163457000000034,                          
  it should.                                                       'Objective':                                    
                                                                   -4.2641635}                                     
  The phone doesn't seem   Objective      Objective                {'Subjective':           True                   
  to accept anything                                               -2.0720863,                                     
  except CBR mp3s                                                  'Objective':                                    
                                                                   -0.13458653999999995}                           
  All three broke within   Objective      Objective                {'Subjective':           True                   
  two months of use.                                               -2.1821797,                                     
                                                                   'Objective':                                    
                                                                   -0.11967964500000007}

...

Classify a product review as either expressing "Subjective" or "Objective" statements. A "Subjective" statement is 
based on personal opinions, feelings, or tastes, and may include evaluative words such as 'great', 'good', 'bad', 
'like', 'dislike', etc. An "Objective" statement is based on factual information about the product, such as its 
features, performance, or occurrences during use, and is not influenced by personal feelings or opinions. It 
typically includes factual descriptions or reports of product performance. Even if the statement seems to express 
dissatisfaction or a negative experience, it should be classified as "Objective" if it provides factual information
about the product's characteristics or performance.

Examples:

Input: The color of this phone is black.
Output: Objective

Input: I think this phone is too expensive for its features.
Output: Subjective

Input: Not loud enough and doesn't turn on like it should.
Output: Objective

Input: All three broke within two months of use.
Output: Objective

Input: I don't like the design of this laptop.
Output: Subjective

Input: The mic is great.
Output: Subjective

Re-apply subjectivity_detection skill to dataset ...

Applying skill: subjectivity_detection

100%|████████████████████████████████| 5/5 [00:00<00:00, 24.79it/s]


Train is done!

Let's see the final instructions:

In [13]:
print(agent.skills)

Total Agent Skills: 1

subjectivity_detection
Classify a product review as either expressing "Subjective" or "Objective" statements. A "Subjective" statement is 
based on personal opinions, feelings, or tastes, and may include evaluative words such as 'great', 'good', 'bad', 
'like', 'dislike', etc. An "Objective" statement is based on factual information about the product, such as its 
features, performance, or occurrences during use, and is not influenced by personal feelings or opinions. It 
typically includes factual descriptions or reports of product performance. Even if the statement seems to express 
dissatisfaction or a negative experience, it should be classified as "Objective" if it provides factual information
about the product's characteristics or performance.

Examples:

Input: The color of this phone is black.
Output: Objective

Input: I think this phone is too expensive for its features.
Output: Subjective

Input: Not loud enough and doesn't turn on like it should.
Output: Objective

Input: All three broke within two months of use.
Output: Objective

Input: I don't like the design of this laptop.
Output: Subjective

Input: The mic is great.
Output: Subjective

... and predictions created by the skill:

In [14]:
agent.run(dataset)

Applying skill: subjectivity_detection

100%|████████████████████████████████| 5/5 [00:00<00:00, 24.62it/s]


,text,ground_truth,subjectivity_detection,score
0,The mic is great.,Subjective,Subjective,"{'Subjective': -0.11115719, 'Objective': -2.25..."
1,Will order from them again!,Subjective,Subjective,"{'Subjective': -0.44846975999999994, 'Objectiv..."
2,Not loud enough and doesn't turn on like it sh...,Objective,Objective,"{'Subjective': -4.4792867, 'Objective': -0.011..."
3,The phone doesn't seem to accept anything exce...,Objective,Objective,"{'Subjective': -4.990218, 'Objective': -0.0068..."
4,All three broke within two months of use.,Objective,Objective,"{'Subjective': -4.19226, 'Objective': -0.01522..."


## Applying learned skills to the real data

Now as we have our Agent with evolved "subjectivity detection" skill, we can apply it to the real dataset without ground truth data:

In [15]:
test_df = pd.DataFrame([
    "Doesn't hold charge.",
    "Excellent bluetooth headset",
    "I love this thing!",
    "VERY DISAPPOINTED."
], columns=['text'])
test_df

,text
0,Doesn't hold charge.
1,Excellent bluetooth headset
2,I love this thing!
3,VERY DISAPPOINTED.


In [16]:
predictions = agent.run(test_df)

Applying skill: subjectivity_detection

100%|████████████████████████████████| 4/4 [00:04<00:00,  1.00s/it]


In [17]:
predictions

,text,subjectivity_detection,score
0,Doesn't hold charge.,Objective,"{'Subjective': -6.241702, 'Objective': -0.0019..."
1,Excellent bluetooth headset,Subjective,"{'Subjective': -0.39792176999999995, 'Objectiv..."
2,I love this thing!,Subjective,"{'Subjective': -0.0008572661999999637, 'Object..."
3,VERY DISAPPOINTED.,Subjective,"{'Subjective': -0.21449489999999996, 'Objectiv..."
